# 数据预处理
:label:`sec_pandas`

为了能用深度学习来解决现实世界的问题，我们经常从预处理原始数据开始，
而不是从那些准备好的张量格式数据开始。
在Python中常用的数据分析工具中，我们通常使用`pandas`软件包。
像庞大的Python生态系统中的许多其他扩展包一样，`pandas`可以与张量兼容。
本节我们将简要介绍使用`pandas`预处理原始数据，并将原始数据转换为张量格式的步骤。
后面的章节将介绍更多的数据预处理技术。

## 读取数据集

举一个例子，我们首先(**创建一个人工数据集，并存储在CSV（逗号分隔值）文件**)
`../data/house_tiny.csv`中。
以其他格式存储的数据也可以通过类似的方式进行处理。
下面我们将数据集按行写入CSV文件中。


In [11]:
import os

dir = os.makedirs(os.path.join('..', 'data'), exist_ok=True)#哦 做一个dir

data_file = os.path.join('..', 'data', 'house_tiny.csv')
print(data_file)
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')  # 列名
    f.write('NA,Pave,127500\n')  # 每行表示一个数据样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

../data/house_tiny.csv


要[**从创建的CSV文件中加载原始数据集**]，我们导入`pandas`包并调用`read_csv`函数。该数据集有四行三列。其中每行描述了房间数量（“NumRooms”）、巷子类型（“Alley”）和房屋价格（“Price”）。


In [25]:
# 如果没有安装pandas，只需取消对以下行的注释来安装pandas
# !pip install pandas
import pandas as pd

data = pd.read_csv(data_file)
data
print(data.dtypes)

NumRooms    float64
Alley        object
Price         int64
dtype: object


## 处理缺失值

注意，“NaN”项代表缺失值。
[**为了处理缺失的数据，典型的方法包括*插值法*和*删除法*，**]
其中插值法用一个替代值弥补缺失值，而删除法则直接忽略缺失值。
在(**这里，我们将考虑插值法**)。

通过位置索引`iloc`，我们将`data`分成`inputs`和`outputs`，
其中前者为`data`的前两列，而后者为`data`的最后一列。
对于`inputs`中缺少的数值，我们用同一列的均值替换“NaN”项。


In [26]:
inputs, outputs = data.iloc[:,  0:2], data.iloc[:, 2] #iloc: index localtion
print(inputs.dtypes)
#inputs = inputs.fillna(inputs.mean()) #所有not a number nan 的 把nan填充不是not a number的均值 也就是3 问题是0和pave怎么会means 是3呢？同一列的均值 搞错了！#inputs.mean()同一列的均值。
numeric_cols = inputs.select_dtypes(include=['number']).columns #所以只有object类型才能选择dtypes. 这里是选了只包含有number的那一列，
inputs[numeric_cols] = inputs[numeric_cols].fillna(inputs[numeric_cols].mean())
# 非数值列用其他方法填充（如众数）
non_numeric_cols = inputs.select_dtypes(exclude=['number']).columns #这个选择了不包含number的那一列。
inputs[non_numeric_cols] = inputs[non_numeric_cols].fillna(inputs[non_numeric_cols].mode().iloc[0])
print(inputs)

NumRooms    float64
Alley        object
dtype: object
   NumRooms Alley
0       3.0  Pave
1       2.0  Pave
2       4.0  Pave
3       3.0  Pave


[**对于`inputs`中的类别值或离散值，我们将“NaN”视为一个类别。**]
由于“巷子类型”（“Alley”）列只接受两种类型的类别值“Pave”和“NaN”，
`pandas`可以自动将此列转换为两列“Alley_Pave”和“Alley_nan”。
巷子类型为“Pave”的行会将“Alley_Pave”的值设置为1，“Alley_nan”的值设置为0。
缺少巷子类型的行会将“Alley_Pave”和“Alley_nan”分别设置为0和1。


In [20]:
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms  Alley_Pave  Alley_nan
0       3.0        True      False
1       2.0        True      False
2       4.0        True      False
3       3.0        True      False


## 转换为张量格式

[**现在`inputs`和`outputs`中的所有条目都是数值类型，它们可以转换为张量格式。**]
当数据采用张量格式后，可以通过在 :numref:`sec_ndarray`中引入的那些张量函数来进一步操作。


In [21]:
import torch

X = torch.tensor(inputs.to_numpy(dtype=float))
y = torch.tensor(outputs.to_numpy(dtype=float))
X, y

(tensor([[3., 1., 0.],
         [2., 1., 0.],
         [4., 1., 0.],
         [3., 1., 0.]], dtype=torch.float64),
 tensor([127500., 106000., 178100., 140000.], dtype=torch.float64))

## 小结

* `pandas`软件包是Python中常用的数据分析工具中，`pandas`可以与张量兼容。
* 用`pandas`处理缺失的数据时，我们可根据情况选择用插值法和删除法。

## 练习

创建包含更多行和列的原始数据集。

1. 删除缺失值最多的列。
2. 将预处理后的数据集转换为张量格式。


In [27]:
import pandas as pd
import numpy as np

# 设置随机种子保证结果可复现
np.random.seed(42)

# 创建5x4的随机数值矩阵（范围0-10）
data = np.random.randint(0, 10, size=(5, 4)).astype(float)

# 为每列随机生成10%-30%的NaN位置
for col in range(4):
    nan_mask = np.random.choice([True, False], size=5, p=[0.2, 0.8])  # 20%概率为NaN
    data[:, col][nan_mask] = np.nan

# 转换为DataFrame并命名列
df = pd.DataFrame(data, columns=['Col1', 'Col2', 'Col3', 'Col4']) #直接把numpy ndarray转化为了object类型 通过在每一列上头加一个key 这个key不参与计算。
print(df.dtypes)
# 打印结果
print("原始数据集（含随机NaN）：")
print(df)

Col1    float64
Col2    float64
Col3    float64
Col4    float64
dtype: object
原始数据集（含随机NaN）：
   Col1  Col2  Col3  Col4
0   NaN   3.0   7.0   NaN
1   NaN   9.0   2.0   6.0
2   7.0   NaN   NaN   NaN
3   7.0   2.0   5.0   NaN
4   1.0   7.0   5.0   1.0


In [31]:
numeric_cols = df.select_dtypes(include=['number']).columns #这个相当于四个都选了。
print(numeric_cols)
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())
print(df)
# 非数值列用其他方法填充（如众数）
non_numeric_cols = df.select_dtypes(include=['number']).columns
df[non_numeric_cols] = df[non_numeric_cols].fillna(df[non_numeric_cols].mode().iloc[0])
print(df)

Index(['Col1', 'Col2', 'Col3', 'Col4'], dtype='object')
   Col1  Col2  Col3  Col4
0   5.0  3.00  7.00   3.5
1   5.0  9.00  2.00   6.0
2   7.0  5.25  4.75   3.5
3   7.0  2.00  5.00   3.5
4   1.0  7.00  5.00   1.0
   Col1  Col2  Col3  Col4
0   5.0  3.00  7.00   3.5
1   5.0  9.00  2.00   6.0
2   7.0  5.25  4.75   3.5
3   7.0  2.00  5.00   3.5
4   1.0  7.00  5.00   1.0


In [34]:
col_means = np.nanmean(data, axis=0)  # axis=0 表示按列计算
# 逐列计算均值（忽略NaN）
col_means = np.nanmean(data, axis=0)  # axis=0 表示按列计算

# 用均值填充每列的NaN
for col in range(data.shape[1]):
    mask = np.isnan(data[:, col])          # 找出当前列的NaN位置
    data[mask, col] = col_means[col]       # 填充均值
print(data)

[[5.   3.   7.   3.5 ]
 [5.   9.   2.   6.  ]
 [7.   5.25 4.75 3.5 ]
 [7.   2.   5.   3.5 ]
 [1.   7.   5.   1.  ]]


[Discussions](https://discuss.d2l.ai/t/1750)
